In [23]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from math import log, ceil

In [14]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas

In [10]:
wiki_list = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            if tokenized_line[-1][-1] == "\n":
                tokenized_line[-1] = tokenized_line[-1][:-1]
            cntr+=1
            wiki_list.append((tokenized_line[-1], []))
            continue
        wiki_list[-1][1].append(nltk.word_tokenize(line))

In [15]:
lemmas = to_lemma_mapping()

In [32]:
positional_index = dd(list)
for i, article in enumerate(wiki_list):
    lines = article[1]
    for words in lines:
        for w in words:
            if w not in lemmas:
                continue
            lemmas_ = lemmas[w]
            for l in lemmas_:
                if len(positional_index[l]) == 0 or positional_index[l][-1] != i+1:
                    positional_index[l].append(i+1)

In [33]:
def compute_vb_size(n, base):
    bit_num = int(log(n, 2)) + 1
    parts = ceil(bit_num / (base-1))
    return parts*base

In [34]:
def compute_gamma_size(n):
    m = int(log(n, 2))
    return 2*m + 1

In [35]:
int32size = 0
vbsize = 0
vb4size = 0
gammasize = 0
for k, v in positional_index.items():
    prev_index = 0
    for index in v:
        int32size += 32
        diff = index - prev_index
#         print(diff)
        prev_index = index
        vbsize += compute_vb_size(diff, 8)
        vb4size += compute_vb_size(diff, 4)      
        gammasize += compute_gamma_size(diff)

In [36]:
print(int32size)

1647220960


In [37]:
print(vbsize)

521425384


In [38]:
print(vb4size)

431271520


In [39]:
print(gammasize)

474647051
